# Model Training

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn

%matplotlib inline

## Data preparation

### Data pre-processing

In [2]:
# Set up the basic paths
data_path = '../../nhl-game-data/'
preproc_data_path = '../data/'

# Read in the data on each game
per_game = pd.read_csv(os.path.join(preproc_data_path, 'per-game.csv'))
per_game['game_id'] = per_game['GameId']
# Read in the data on each event within each game
game_plays = pd.read_csv(os.path.join(data_path, 'game_plays.csv'))
# Read in the data on summary statistics within each game for both teams
game_teams = pd.read_csv(os.path.join(data_path, 'game_teams_stats.csv'))

In [5]:
# Gather all of the home and away games for each team
g_home = game_teams[game_teams['HoA'] == 'home'][['game_id', 'team_id', 'goals']].copy()
g_away = game_teams[game_teams['HoA'] == 'away'][['game_id', 'team_id', 'goals']].copy()

# Combine them into a wider dataframe
game_summary = pd.merge(g_home, g_away, on='game_id', suffixes=('_home', '_away'))

In [ ]:
# We only want "action" events, where there is a doer and receiver team, i.e. not things like timeout
action_plays = game_plays[~game_plays['team_id_for'].isna() & ~game_plays['team_id_against'].isna()].copy()

# Create a continuous time measure out of the period & periodTime variables
action_plays['time'] = action_plays['period'] - 1.0 + \
                       action_plays['periodTime'] / (action_plays['periodTime'] + action_plays['periodTimeRemaining'])

In [8]:
# Add the game-level covariates to the event dataframe created above
events = pd.merge(action_plays, game_summary[['game_id', 'team_id_home', 'team_id_away']] , on='game_id')
# Append a 1 (Home) and -1 (Away) indicator for which side initated the action
events['h_a_sign'] = (events['team_id_for'].astype(int) == events['team_id_home']).astype(float) * 2.0 - 1.0

# Remove unused variables from the data, such as play ids, old time measures, and player locations
# NOTE: In future iterations we can refine the model to also take into account player locations
remove_cols = ['play_id', 'play_num', 'team_id_for', 'team_id_against', 'period', 
               'periodTime', 'periodTimeRemaining', 'dateTime', 
               'description', 'rink_side', 'secondaryType', 'x', 'y', 'st_x', 'st_y',
               'team_id_home', 'team_id_away']
events.drop(remove_cols, axis=1, inplace=True)

In [ ]:
# Merge the game information with summary measures of the relevant event types
tmp = pd.merge(per_game, game_summary[['game_id', 'team_id_home', 'team_id_away']], on='game_id')
# Multiply each summary measure by 1 (if Home) or -1 (if Away)
tmp.iloc[:, 2:-3] *= (tmp['TeamId'] == tmp['team_id_home']).map({True: 1.0, False: -1.0})[:, None]
# Drop the duplicate columns
tmp.drop(['TeamId', 'GameId', 'team_id_home', 'team_id_away'], axis=1, inplace=True)

# Aggregate all of the relevant measures by game and sum them up to create a history for each of the measures
# NOTE: Since Home == 1, Away == -1, we are calculating the difference between home & away performance for each game
game_history = tmp.groupby('game_id').sum()
# Normalize the history measure
game_history = (game_history - game_history.mean()) / (game_history.std())

In [12]:
# Gather all of the game ids for which we have event data for
all_game_ids = sorted(events['game_id'].unique())
# Drop all games for which we DO NOT have event data for
game_history = game_history[game_history.index.isin(all_game_ids)].copy()
game_summary = game_summary[game_summary['game_id'].isin(all_game_ids)].copy()

In [13]:
# And finally, set game_id as the index for the data
events.set_index('game_id', inplace=True)
game_summary.set_index('game_id', inplace=True)

### Creating Torch Dataset

In [14]:
# Count the number of event types; We add 1 for the <GAME_START> event.
NUM_EVENT_TYPES = 1 + events['event'].unique().size
# Create a mapping from the event type to its unique id
EVENT_TYPE_DICT = {e_type: i + 1 for i, e_type in enumerate(events['event'].unique())}
# Count the number of history measures we have
HIST_DIM = game_history.shape[1]

In [ ]:
class GameEventsDataset(torch.utils.data.Dataset):
    def __init__(self, game_ids, game_history, game_summary, events):
        self.game_ids = game_ids
        self.game_history = game_history.copy()
        self.game_summary = game_summary.copy()
        self.events = events.copy()
                
    def __getitem__(self, index):
        # Select the appropriate game
        game_id = self.game_ids[index]
        
        # Game statistics for the given game
        s = self.game_summary.loc[game_id]
        team_h = s['team_id_home']
        team_a = s['team_id_away']
        g_h = s['goals_home']
        g_a = s['goals_away']

        # NOTE: Score is our outcome of interest
        score = torch.tensor(float(g_h - g_a))
        # But we also care about which team is winning: 0 (Home is losing), 1 (Tie), 2 (Home is winning)
        target = (torch.sign(score) + 1.0).long()
        
        # Game history tensor
        h = torch.Tensor(self.game_history.loc[game_id])

        # Subset all of the events to the game of interest
        e = self.events.loc[game_id]
        # Create an event dataframe with dimensions: (# of events in game) x (# of event types)
        e_X = np.zeros((e.shape[0], NUM_EVENT_TYPES))
        # One-hot encode the events in the given game
        e_X[np.arange(e_X.shape[0]), e['event'].map(EVENT_TYPE_DICT)] = 1.0
        # Multiply the one-hot encoding by the 1, -1 Home/Away indicator
        e_X *= e['h_a_sign'][:, None]
        # And finally, create the event tensor
        X = np.array(e_X)
        X = torch.Tensor(X)

        # Also create a tensor out of the continuous time measure from before
        t = np.array(e['time'])
        t = torch.Tensor(t)
        
        # In addition to the overall score measure, we also want one at each time point
        s_t = torch.Tensor(np.array(e['goals_home'] - e['goals_away']))
        
        # For each game, return the history for both teams, time, events,
        #   game result, final score, score at each of the observed time points
        return h, t, X, target, score, s_t
        
    def __len__(self):
        return len(self.game_ids)

In [18]:
# Split up the data into training and test games
N = len(all_game_ids)
# First 80% of all games are used as training data, rest is test
train_game_ids = all_game_ids[:int(0.8 * N)]
test_game_ids = all_game_ids[int(0.8 * N):]

# And create the PyTorch training and test dataset from the above ids
train_ds = GameEventsDataset(train_game_ids, game_history, game_summary, events)
test_ds = GameEventsDataset(test_game_ids, game_history, game_summary, events)

In [ ]:
# Create the PyTorch training and test data loader from the above training and test data
train_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=1,
    num_workers=5,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_ds,
    batch_size=1,
    num_workers=5,
    shuffle=False
)

## Model Definition of MD-RNN

### Mixture Density Layer

In [ ]:
class MDLayer(nn.Module):
    def __init__(self, input_size, K=5):
        super(MDLayer, self).__init__()
        
        # Mixture probabilities for K Normals
        self.linear_pi = nn.Linear(input_size, K)
        # Means and variances of Normal densities
        self.linear_mu = nn.Linear(input_size, K)
        self.linear_var = nn.Linear(input_size, K)
        
        # Initialize the intercept for the means to be evenly spaced between -5 and 5
        self.linear_mu.bias.data.copy_(torch.linspace(-5, 5, K, dtype=torch.float32))
        # Initialize the intercept for the variances to be -0.5 for all mixtures.
        self.linear_var.bias.data.fill_(-0.5)
        
    def forward(self, x):
        logits_pi = self.linear_pi(x)
        mu = self.linear_mu(x)
        # NOTE: Working with log-variance, so keep it between ~0.1 and ~7.4
        log_var = torch.clamp(self.linear_var(x), min=-2.0, max=2.0)
        return logits_pi, mu, log_var

### Mixture Density RNN

In [397]:
class MDRNN(nn.Module):
    def __init__(self, h_in_dim, e_in_dim, hidden_size=100, K=5):
        super(MDRNN, self).__init__()
        # History transformation layers
        self.history_transform = nn.Sequential(
            nn.Linear(h_in_dim, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, hidden_size),
            nn.Tanh()
        )
        
        # Mixture density layers
        self.mdn_net = nn.Sequential(
            nn.Linear(hidden_size, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 50),
            nn.Tanh(),
            MDLayer(input_size=50, K=K)
        )
        
        # Layers to transform further to get them ready for auxiliary output calculation
        self.aux_z = nn.Sequential(
            nn.Linear(hidden_size, 100),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, 50),
            nn.Tanh(),
        )
        
        # Auxiliary output 1: Game outcome (Home Loss, Tie, Home Win)
        self.aux_o = nn.Linear(50, 3)
        
        # Auxiliary output 2: Current score (Continuous)
        self.aux_s_t = nn.Linear(50, 1)
        
        # Recurrent layer
        self.gru = nn.GRU(input_size=e_in_dim,
                          hidden_size=hidden_size,
                          num_layers=1,
                          batch_first=True)
        
    def apply_aux(self, x):
        # Apply the initial auxiliary transformation
        z = self.aux_z(x)
        # Predict the outcome probabilities of the game
        o_logits = self.aux_o(z)
        # And the current score
        s_t = self.aux_s_t(z).squeeze(-1)
        # Return both the outcome probabilities and score
        return o_logits, s_t
    
    def process_history(self, h):
        # Transform the history
        h_features = self.history_transform(h)
        # Estimate the mixture density parameters
        h_out = self.mdn_net(h_features)
        # And the auxiliary outputs for the history
        h_aux = self.apply_aux(h_features)
        # Return all three of those
        return h_out, h_features, h_aux
        
    def forward(self, h, t, X):
        # First step is to process the history up until this game
        h_out, h_features, h_aux = self.process_history(h)
        
        # The recurrent input is the one-hot encoded event types
        gru_in = X
        # The initial state are the transformed history features
        gru_state_0 = h_features.unsqueeze(1)
        
        # Pass the data through the GRU layer
        gru_out, gru_state_n = self.gru(gru_in, gru_state_0)
        
        # Estimate the mixture density parameters
        e_out = self.mdn_net(gru_out)
        # And the auxiliary outputs
        e_aux = self.apply_aux(gru_out)

        # Return the initial and post-GRU mixture density parameters, and auxiliary outputs
        return h_out, e_out, h_aux, e_aux
        

### Loss Functions

#### Helper Functions

In [398]:
# Helper function to calculate the log-likelihood of a Normal
def normal_log_pdf(mu, log_var, score):
    log_C = -0.5 * (np.log(2.0 * np.pi) + log_var)
    
    shape = [score.size(0)] + [1] * max(mu.ndim - 2, 0) + [1]
    score_view = score.view(*shape)
    return log_C - 0.5 * ((mu - score_view) ** 2 / torch.exp(log_var))

# Helper function to calculate the log-likelihood of the full mixture density model
def mdn_log_likelihood(out, score):
    logits_pi, mu, log_var = out
    log_normal = normal_log_pdf(mu, log_var, score)
    
    log_pi_norm = torch.logsumexp(logits_pi, dim=-1, keepdim=True)
    log_pi = logits_pi - log_pi_norm
    
    log_v = log_pi + log_normal
    log_ll = torch.logsumexp(log_v, dim=-1)
    return log_ll

# Calculate the cross entropy loss for the game outcomes for the history
def compute_h_aux_loss(h_aux, target):
    o_logits, s_t_pred = h_aux
    return nn.functional.cross_entropy(o_logits, target)

def compute_e_aux_loss(e_aux, target, s_t, score_weight=1.0):
    # Calculate the cross entropy loss for the game outcomes for the events
    def event_loss(out, target):
        p = out.view(-1, 3)
        return nn.functional.cross_entropy(p, target.repeat(p.size(0)))
    o_logits, s_t_pred = e_aux
    # Cross-Entropy loss for the outcome probabilities
    t_loss = event_loss(o_logits, target)
    # MSE loss for the current score difference
    s_loss = nn.functional.mse_loss(s_t_pred, s_t)
    return t_loss + s_loss * score_weight

#### MD-RNN Loss Function

In [ ]:
# The custom loss function for our network
def get_mdn_loss_fn(pretraining=False, history_weight=2.0, score_weight=1.0):
    def loss_fn(model, data):
        # Parse out the inputs from the data
        h, t, X, target, score, s_t = data
        # Get the outputs from the model
        h_out, e_out, h_aux, e_aux = model(h, t, X)

        # Want the negative log-likelihoods for the mixture densities
        h_nll = -1.0 * mdn_log_likelihood(h_out, score)
        e_nll = -1.0 * mdn_log_likelihood(e_out, score)
        
        # Calculate the losses for the auxiliary outputs
        h_aux_loss = compute_h_aux_loss(h_aux, target)
        e_aux_loss = compute_e_aux_loss(e_aux, target, s_t, score_weight=score_weight)
        
        # Don't include the MD negative log-likelihood in the pre-training
        nll_weight = 0.0 if pretraining else 1.0

        # Calculate the total losses for both the pre-GRU and post-GRU parameters
        e_total = e_aux_loss + nll_weight * torch.mean(e_nll)
        h_total = h_aux_loss + nll_weight * h_nll

        # And return a weighted sum of them
        return e_total + history_weight * h_total
    return loss_fn

### MD-RNN Evaluation Function

In [399]:
# Helper function to see whether the prediction at time t is correct
def correct_at_t(t, logits_series, target, cut_off=1.0):
    # Find the last event index before the cut_off
    ind = torch.max(t * (t < cut_off).float(), dim=-1)[1]
    # Gather all of the logits at the event right before the cut_off
    logits_at_t = logits_series[np.arange(t.size(0)), ind, :]
    # Get the outcome prediction
    p = torch.max(logits_at_t, dim=-1)[1]
    # And check whether it was correct
    return torch.eq(p, target).sum()

def evaluate_mdrnn(model, loader):
    # Tell the model that we are evauluating
    model.eval()

    # Initialize the negative log-likelihoods
    h_nll_sum = 0.0
    e_nll_sum = 0.0
    
    # Initialize the auxiliary losses
    h_aux_loss_sum = 0.0
    e_aux_loss_sum = 0.0

    # Initialize the number of correct predictions outcome predictions
    h_correct = 0.0
    e_correct_1 = 0.0
    e_correct_2 = 0.0
    # And the MSE for the current score
    e_mse_sum = 0.0

    # DO NOT attach the gradients
    with torch.no_grad():
        # For each observation in the data
        for data in loader:
            # Parse out the input and outputs
            data = tuple(tensor.cuda() for tensor in data)
            h, t, X, target, score, s_t = data
            
            # Calculate the negative log-likelihood for the history MD model
            h_out, e_out, h_aux, e_aux = model(h, t, X)
            h_nll = -1.0 * mdn_log_likelihood(h_out, score)
            h_nll_sum += h_nll.item() * h.size(0)
            
            # Calculate the negative log-likelihood for the event MD model
            e_nll = -1.0 * mdn_log_likelihood(e_out, score)
            e_nll_sum += torch.mean(e_nll).item() * h.size(0)
            
            # Calculate the auxiliary loss for the history outcome model
            h_aux_loss = compute_h_aux_loss(h_aux, target)
            h_aux_loss_sum += h_aux_loss.item() * h.size(0)
            
            # Calculate the auxiliary loss for the event outcome model
            e_aux_loss = compute_e_aux_loss(e_aux, target, s_t)
            e_aux_loss_sum += e_aux_loss.item() * h.size(0)
            
            # Extract the predicted game outcome for the history model
            h_label = torch.max(h_aux[0], dim=-1)[1]
            # Add 1 if it got the outcome correct
            h_correct += torch.eq(h_label, target).sum()
            
            # Check whether the outcome for before the first period is correct
            e_correct_1 += correct_at_t(t, e_aux[0], target, cut_off=1.0)
            # Check whether the outcome for before the second period is correct
            e_correct_2 += correct_at_t(t, e_aux[0], target, cut_off=2.0)
            # Calculate the MSE loss for the current score difference
            e_mse_sum += nn.functional.mse_loss(e_aux[1], s_t).item() * h.size(0)
    
    # Tell the model that we are no longer evaluating, but training again
    model.train()
    N = len(loader.dataset)
    # Return a nice summary of all of the measures above
    return {
        'h_nll': h_nll_sum / N,
        'e_nll': e_nll_sum / N,
        'h_aux_loss': h_aux_loss_sum / N,
        'e_aux_loss': e_aux_loss_sum / N,
        'acc_0': h_correct / N * 100.0,
        'acc_1': e_correct_1 / N * 100.0,
        'acc_2': e_correct_2 / N * 100.0,
        'e_rmse': np.sqrt(e_mse_sum / N),
    }

## Training Loop

### General Training Loop Function

In [ ]:

import time

def traininig_loop(model, optimizer, train_loader, test_loader, loss_fn, eval_fn, num_steps=10000, eval_interval=1000):
    # Inform the model, that we are training it and not evaluating
    model.train()
    loader_it = iter(train_loader)

    K = 0
    loss_queue = []
    t_0 = time.time()
    # For each iteration
    for step in range(num_steps):
        # Continuously load in the data
        try:
            data = next(loader_it)
        # And restart once we run out
        except StopIteration:
            loader_it = iter(train_loader)
            data = next(loader_it)
        
        # h, t, X, target, score, s_t
        data = tuple(tensor.cuda() for tensor in data)
        
        # Calculate the loss of the model with the given data
        loss = loss_fn(model, data)

        # Zero-out your gradient before updating the parameters
        optimizer.zero_grad()
        # Back propagate to store the new gradients for all parameters
        loss.backward()
        # Do gradient-descent based on the new gradients to update parameter values
        optimizer.step()
        
        # Get loss from given iteration
        loss_v = loss.item()
        # Increment the time-averaging step counter
        K += 1
        # If we reached the evaluation interval length, pop the oldest losses
        if len(loss_queue) == eval_interval:
            loss_queue.pop(0)
        # Keep appending the loss to the queue
        loss_queue.append(loss_v)
        
        # Print out training stats at the given eval_interval speed
        if step % eval_interval == 0 or step == num_steps - 1:
            # Evaluate the model
            eval_res = eval_fn(model, test_loader)
            # And create the measures of interest as a string
            eval_res_strings = ['%s: %.2f' % (name, val) for name, val in eval_res.items()]
            eval_str = ' '.join(eval_res_strings)
            
            # Calculate the time it took to take K steps
            dt = (time.time() - t_0) / K

            # Print out the step count, mean of the losses so far, time/iteration, and the evaluation results
            print('Step: %05d. Loss: %.4f Time/it: %.2f sec.\n%s' % (step, np.mean(loss_queue), dt, eval_str))
            # Print a new line
            print()

            # Start measuring time and steps from scratch again
            t_0 = time.time()
            K = 0

### Train the MD-RNN

In [400]:
# Instantiate the mixture density RNN with 7 Normals
mdrnn = MDRNN(h_in_dim=HIST_DIM, e_in_dim=NUM_EVENT_TYPES, K=7)
mdrnn.cuda()

MDRNN(
  (history_transform): Sequential(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Tanh()
  )
  (mdn_net): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Tanh()
    (4): MDLayer(
      (linear_pi): Linear(in_features=50, out_features=7, bias=True)
      (linear_mu): Linear(in_features=50, out_features=7, bias=True)
      (linear_var): Linear(in_features=50, out_features=7, bias=True)
    )
  )
  (aux_z): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Tanh()
  )
  (aux_o): Linear(in_features=50, out_features=3, bias=True)
  (aux_s_t): L

In [402]:
# Train the network
traininig_loop(
    mdrnn,
    optimizer=torch.optim.Adam(mdrnn.parameters()),
    train_loader=train_loader,
    test_loader=test_loader,
    loss_fn=get_mdn_loss_fn(pretraining=False, score_weight=1.0),
    eval_fn=evaluate_mdrnn,
    num_steps=25000,
    eval_interval=1000)

Step: 00000. Loss: 20.1904 Time/it: 21.17 sec.
h_nll: 2.49 e_nll: 2.49 h_aux_loss: 1.14 e_aux_loss: 3.74 acc_0: 6.63 acc_1: 6.54 acc_2: 6.54 e_rmse: 1.61

Step: 01000. Loss: 11.9844 Time/it: 0.04 sec.
h_nll: 2.34 e_nll: 2.24 h_aux_loss: 0.89 e_aux_loss: 2.33 acc_0: 42.73 acc_1: 61.09 acc_2: 67.27 e_rmse: 1.23

Step: 02000. Loss: 10.8938 Time/it: 0.04 sec.
h_nll: 2.30 e_nll: 2.23 h_aux_loss: 0.91 e_aux_loss: 2.67 acc_0: 50.78 acc_1: 60.16 acc_2: 61.27 e_rmse: 1.34

Step: 03000. Loss: 9.8888 Time/it: 0.04 sec.
h_nll: 2.27 e_nll: 2.02 h_aux_loss: 0.89 e_aux_loss: 0.99 acc_0: 50.96 acc_1: 64.12 acc_2: 74.79 e_rmse: 0.53

Step: 04000. Loss: 9.4214 Time/it: 0.04 sec.
h_nll: 2.30 e_nll: 2.16 h_aux_loss: 0.90 e_aux_loss: 1.86 acc_0: 49.62 acc_1: 62.61 acc_2: 71.10 e_rmse: 1.04

Step: 05000. Loss: 9.3849 Time/it: 0.04 sec.
h_nll: 2.29 e_nll: 1.98 h_aux_loss: 0.91 e_aux_loss: 0.77 acc_0: 49.58 acc_1: 64.34 acc_2: 75.19 e_rmse: 0.25

Step: 06000. Loss: 9.2065 Time/it: 0.04 sec.
h_nll: 2.28 e_nll: